In [1]:
import duckdb
import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Define the directory containing your CSV files
# data_dir = '/kaggle/input/sec-cameco/' if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ else '../data/tables_google_ms_nvidia/'
data_dir = '/kaggle/input/sec-cameco/' if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ else '../data/tables_all/'

for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


def print_duckdb_schema(conn: duckdb.DuckDBPyConnection, table_name: str):
  """Prints the schema of a specified table in a DuckDB database.

  Args:
    conn: An active DuckDB connection object.
    table_name: The name of the table whose schema to print.
  """
  try:
    schema_info = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    if not schema_info:
      print(f"Table '{table_name}' not found.")
      return

    print(f"Schema for table: {table_name}")
    print("-" * (16 + len(table_name)))
    print(f"{'Column ID':<10} {'Name':<20} {'Type':<15} {'NotNull':<8} {'PrimaryKey':<12} {'Default':<20}")
    print("-" * 85)
    for column in schema_info:
      cid, name, dtype, notnull, pk, default = column
      print(f"{cid:<10} {name:<20} {dtype:<15} {bool(notnull):<8} {bool(pk):<12} {str(default):<20}")
    print("-" * 85)

  except duckdb.CatalogException as e:
    print(f"Error: {e}")
    print(f"Could not retrieve schema for table '{table_name}'.")

../data/tables_all/dim.csv
../data/tables_all/tag.csv
../data/tables_all/pre.csv
../data/tables_all/num.csv
../data/tables_all/ren.csv
../data/tables_all/sub.csv
../data/tables_all/cal.csv
../data/tables_all/txt.csv


In [2]:


sec_pk = {'sub': 'adsh', 'tag': 'tag, version', 'ren': 'adsh, report', 'pre': 'adsh, report, line', 'cal': 'adsh, grp, arc', 'dim': 'dimhash'}
# sec_date_column =  {'sub': ['changed', 'period', 'filed', 'floatdate'], 'num': ['ddate'], 'txt': ['ddate']} 
sec_date_column =  {'sub': ['changed', 'period', 'filed', 'floatdate'], 'num': ['ddate'], 'txt': ['ddate']} 
# Create an in-memory DuckDB connection
# con = duckdb.connect(database=':memory:', read_only=False)
con = duckdb.connect(database='../kaggle.db')

# List all files in the specified directory
all_files = os.listdir(data_dir)

# Filter for CSV files (you might need to adjust the extension if your files are different)
csv_files = [f for f in all_files if f.endswith('.csv')]

# Iterate through the CSV files and load them into DuckDB tables
for file_name in csv_files:
    file_path = os.path.join(data_dir, file_name)
    table_name = os.path.splitext(file_name)[0]  # Use the filename (without extension) as the table name

    try:
        # Option 1: Use pandas to read the CSV file into a DataFrame
        #df = pd.read_csv(file_path)

        # Load the DataFrame into a DuckDB table
        #con.register(table_name, df)  # Register the DataFrame as a view

        # Option 2: Using COPY FROM (Faster for large files)
        con.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM read_csv('{file_path}', AUTO_DETECT=TRUE);""")
        if table_name in sec_date_column:
            strptimes: str = ""
            for col in sec_date_column[table_name]:
                # strptimes += f"STRPTIME(CAST({col} AS VARCHAR), '%Y%m%d')::DATE AS {col}, \n"
                # sql_alter = f"ALTER TABLE {table_name} ALTER COLUMN {col} TYPE DATE USING STRPTIME(CAST({col} AS VARCHAR), '%Y%m%d')::DATE;" 
                # sql_alter = f"ALTER TABLE {table_name} ALTER COLUMN {col} TYPE DATE USING STRPTIME(CAST(FLOOR({col}) AS VARCHAR), '%Y%m%d')::DATE;"
                sql_alter = f"ALTER TABLE {table_name} ALTER COLUMN {col} TYPE DATE USING STRPTIME(CAST({col} AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;"
                print(sql_alter)
                con.execute(sql_alter)
            # sql_command = f"""SELECT *, {strptimes.rstrip(', \n')} FROM {table_name};"""


        if table_name in sec_pk:
            # Add primary key constraint
            sql_pk = f"ALTER TABLE {table_name} ADD PRIMARY KEY ({sec_pk[table_name]});"
            print(sql_pk)
            con.execute(sql_pk)


        print(f"Loaded '{file_name}' into DuckDB view '{table_name}'")

    except Exception as e:
        print(f"Error loading '{file_name}': {e}")

# Now you can query the data in DuckDB using SQL
# For example, to select the first few rows of one of your tables:
# table_to_query = csv_files[0].split('.')[0] # Get the name of the first table
# result = con.execute(f"SELECT * FROM {table_to_query} LIMIT 5;").fetchdf()
# print(result)

ALTER TABLE dim ADD PRIMARY KEY (dimhash);
Loaded 'dim.csv' into DuckDB view 'dim'
ALTER TABLE tag ADD PRIMARY KEY (tag, version);
Loaded 'tag.csv' into DuckDB view 'tag'
ALTER TABLE pre ADD PRIMARY KEY (adsh, report, line);
Loaded 'pre.csv' into DuckDB view 'pre'
ALTER TABLE num ALTER COLUMN ddate TYPE DATE USING STRPTIME(CAST(ddate AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
Loaded 'num.csv' into DuckDB view 'num'
ALTER TABLE ren ADD PRIMARY KEY (adsh, report);
Loaded 'ren.csv' into DuckDB view 'ren'
ALTER TABLE sub ALTER COLUMN changed TYPE DATE USING STRPTIME(CAST(changed AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN period TYPE DATE USING STRPTIME(CAST(period AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN filed TYPE DATE USING STRPTIME(CAST(filed AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ALTER COLUMN floatdate TYPE DATE USING STRPTIME(CAST(floatdate AS BIGINT)::VARCHAR, '%Y%m%d')::DATE;
ALTER TABLE sub ADD PRIMARY KEY (adsh);
Loaded '

In [3]:
query_table = 'sub'
result = con.execute(f"SELECT * FROM {query_table};")
print(result.fetchdf())

print_duckdb_schema(con, query_table)


                     adsh      cik                      name     sic  \
0    0000891618-09-000150  1164727  NEWMONT MINING CORP /DE/  1040.0   
1    0000950123-09-024448  1164727  NEWMONT MINING CORP /DE/  1040.0   
2    0000950123-09-054293  1164727  NEWMONT MINING CORP /DE/  1040.0   
3    0000950123-10-016613  1164727  NEWMONT MINING CORP /DE/  1040.0   
4    0001047469-10-002778     2809    AGNICO EAGLE MINES LTD  1040.0   
..                    ...      ...                       ...     ...   
269  0001047469-14-006866     2809    AGNICO EAGLE MINES LTD  1040.0   
270  0001564590-14-004746  1164727  NEWMONT MINING CORP /DE/  1040.0   
271  0001564590-15-000777  1164727  NEWMONT MINING CORP /DE/  1040.0   
272  0001564590-15-002819  1164727  NEWMONT MINING CORP /DE/  1040.0   
273  0001558370-15-001173  1164727  NEWMONT MINING CORP /DE/  1040.0   

    countryba stprba             cityba    zipba  \
0          US     CO  GREENWOOD VILLAGE    80222   
1          US     CO  GREENWOOD

In [ ]:
con.close()